<a href="https://colab.research.google.com/github/nnamdisammie/Hamoye/blob/main/Hamoye_regression.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [6]:
Title = pd.read_csv('energy.csv')

In [7]:
Title

,date,Appliances,lights,T1,RH_1,T2,RH_2,T3,RH_3,T4,RH_4,T5,RH_5,T6,RH_6,T7,RH_7,T8,RH_8,T9,RH_9,T_out,Press_mm_hg,RH_out,Windspeed,Visibility,Tdewpoint,rv1,rv2
0,2016-01-11 17:00:00,60,30,19.890000,47.596667,19.200000,44.790000,19.790000,44.730000,19.000000,45.566667,17.166667,55.200000,7.026667,84.256667,17.200000,41.626667,18.2000,48.900000,17.033333,45.5300,6.600000,733.5,92.000000,7.000000,63.000000,5.300000,13.275433,13.275433
1,2016-01-11 17:10:00,60,30,19.890000,46.693333,19.200000,44.722500,19.790000,44.790000,19.000000,45.992500,17.166667,55.200000,6.833333,84.063333,17.200000,41.560000,18.2000,48.863333,17.066667,45.5600,6.483333,733.6,92.000000,6.666667,59.166667,5.200000,18.606195,18.606195
2,2016-01-11 17:20:00,50,30,19.890000,46.300000,19.200000,44.626667,19.790000,44.933333,18.926667,45.890000,17.166667,55.090000,6.560000,83.156667,17.200000,41.433333,18.2000,48.730000,17.000000,45.5000,6.366667,733.7,92.000000,6.333333,55.333333,5.100000,28.642668,28.642668
3,2016-01-11 17:30:00,50,40,19.890000,46.066667,19.200000,44.590000,19.790000,45.000000,18.890000,45.723333,17.166667,55.090000,6.433333,83.423333,17.133333,41.290000,18.1000,48.590000,17.000000,45.4000,6.250000,733.8,92.000000,6.000000,51.500000,5.000000,45.410389,45.410389
4,2016-01-11 17:40:00,60,40,19.890000,46.333333,19.200000,44.530000,19.790000,45.000000,18.890000,45.530000,17.200000,55.090000,6.366667,84.893333,17.200000,41.230000,18.1000,48.590000,17.000000,45.4000,6.133333,733.9,92.000000,5.666667,47.666667,4.900000,10.084097,10.084097
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19730,2016-05-27 17:20:00,100,0,25.566667,46.560000,25.890000,42.025714,27.200000,41.163333,24.700000,45.590000,23.200000,52.400000,24.796667,1.000000,24.500000,44.500000,24.7000,50.074000,23.200000,46.7900,22.733333,755.2,55.666667,3.333333,23.666667,13.333333,43.096812,43.096812
19731,2016-05-27 17:30:00,90,0,25.500000,46.500000,25.754000,42.080000,27.133333,41.223333,24.700000,45.590000,23.230000,52.326667,24.196667,1.000000,24.557143,44.414286,24.7000,49.790000,23.200000,46.7900,22.600000,755.2,56.000000,3.500000,24.500000,13.300000,49.282940,49.282940
19732,2016-05-27 17:40:00,270,10,25.500000,46.596667,25.628571,42.768571,27.050000,41.690000,24.700000,45.730000,23.230000,52.266667,23.626667,1.000000,24.540000,44.400000,24.7000,49.660000,23.200000,46.7900,22.466667,755.2,56.333333,3.666667,25.333333,13.266667,29.199117,29.199117
19733,2016-05-27 17:50:00,420,10,25.500000,46.990000,25.414000,43.036000,26.890000,41.290000,24.700000,45.790000,23.200000,52.200000,22.433333,1.000000,24.500000,44.295714,24.6625,49.518750,23.200000,46.8175,22.333333,755.2,56.666667,3.833333,26.166667,13.233333,6.322784,6.322784


Question 12
From the dataset, fit a linear model on the relationship between the temperature in the living room in Celsius (x = T2) and the temperature outside the building (y = T6). What is the R^2 value in two d.p.?



In [8]:
Title['T2']

0        19.200000
1        19.200000
2        19.200000
3        19.200000
4        19.200000
           ...    
19730    25.890000
19731    25.754000
19732    25.628571
19733    25.414000
19734    25.264286
Name: T2, Length: 19735, dtype: float64

In [9]:
Title['T6']

0         7.026667
1         6.833333
2         6.560000
3         6.433333
4         6.366667
           ...    
19730    24.796667
19731    24.196667
19732    23.626667
19733    22.433333
19734    21.026667
Name: T6, Length: 19735, dtype: float64

In [10]:
from sklearn.linear_model import LinearRegression
reg = LinearRegression()
X= np.array(Title['T2'])
y= np.array(Title['T6'])

In [11]:
X = X.reshape(-1,1)
y= y.reshape(-1,1)

In [13]:
reg.fit(X,y)

LinearRegression()

In [15]:
from sklearn.metrics import r2_score
score = r2_score(X,y)

In [16]:
score

-35.393237547324794

Question 13
Normalize the dataset using the MinMaxScaler after removing the following columns: [“date”, “lights”]. The target variable is “Appliances”. Use a 70-30 train-test set split with a random state of 42 (for reproducibility). Run a multiple linear regression using the training set and evaluate your model on the test set. Answer the following questions:

What is the Mean Absolute Error (in two decimal places)?



In [17]:
Title = Title.drop(columns = ['date'])

In [20]:
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
scaler = MinMaxScaler()
normalised_df = pd.DataFrame(scaler.fit_transform(Title), columns=Title.columns)
features_df = normalised_df.drop(columns=['lights', 'Appliances'])
target = normalised_df['Appliances']

X_train, X_test, y_train, y_test = train_test_split(features_df, target, test_size =0.3, random_state = 42)

In [22]:
reg.fit(X_train, y_train)

LinearRegression()

In [23]:
predicted_value = reg.predict(X_test)

In [24]:
from sklearn.metrics import mean_absolute_error
mae = mean_absolute_error(y_test, predicted_value)
mae

0.05013362397742959

Question 14
What is the Residual Sum of Squares (in two decimal places)?



In [25]:
Residual_sum = np.sum(np.square(y_test - predicted_value))
print(round(Residual_sum, 2))

45.35


Question 15
What is the Root Mean Squared Error (in three decimal places)?



In [26]:
RME = np.sqrt(mean_squared_error(y_test, predicted_value))
print(round(RME, 3))

0.088


Question 16
What is the Coefficient of Determination (in two decimal places)?



In [27]:
from sklearn.metrics import r2_score
r2_score = r2_score(y_test, predicted_value)
print(round(r2_score, 2))

0.15


Question 17
Obtain the feature weights from your linear model above. Which features have the lowest and highest weights respectively?







In [29]:
def get_weights(model, feat, col_name):
  weights = pd.Series(model.coef_, feat.columns).sort_values()
  weights = pd.DataFrame(weights).reset_index()
  weights.columns = ['Features', col_name]
  weights[col_name].round(3)
  return weights

model = reg.fit(X_train, y_train)
linear_model_weights = get_weights(model, X_train, 'Linear_Model_Weight')

In [30]:
linear_model_weights

,Features,Linear_Model_Weight
0,RH_2,-0.456698
1,T_out,-0.321860
2,T2,-0.236178
3,T9,-0.189941
4,RH_8,-0.157595
5,RH_out,-0.077671
6,RH_7,-0.044614
7,RH_9,-0.039800
8,T5,-0.015657
9,T1,-0.003281


Question 18
Train a ridge regression model with an alpha value of 0.4. Is there any change to the root mean squared error (RMSE) when evaluated on the test set?

In [31]:
from sklearn.linear_model import Ridge
ridge_reg = Ridge(alpha=0.4)
ridge_reg.fit(X_train, y_train)

Ridge(alpha=0.4)

Question 19
Train a lasso regression model with an alpha value of 0.001 and obtain the new feature weights with it. How many of the features have non-zero feature weights?







In [32]:
from sklearn.linear_model import Lasso
lasso = Lasso(alpha=0.001)
lasso_mdl =  lasso.fit(X_train, y_train)

In [33]:
lasso_model_weights = get_weights(lasso_mdl, X_train, 'Linear_Model_Weight')
lasso_model_weights

,Features,Linear_Model_Weight
0,RH_out,-0.049557
1,RH_8,-0.000110
2,T1,0.000000
3,Tdewpoint,0.000000
4,Visibility,0.000000
5,Press_mm_hg,-0.000000
6,T_out,0.000000
7,RH_9,-0.000000
8,T9,-0.000000
9,T8,0.000000


Question 20
What is the new RMSE with the lasso regression? (Answer should be in three (3) decimal places)



In [34]:
lasso_prediction = lasso.predict(X_test)
rmse = np.sqrt(mean_squared_error(y_test, lasso_prediction))
print(round(rmse, 3))

0.094
